In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('drive', force_remount=True)
from pathlib import Path
from tqdm import tqdm

!pip install surprise

from surprise import SVD #so surprise is a python scikit for building and analyzing rocommender systems - from first research this is exactly what Im going for
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

Mounted at drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
user_df = pd.read_csv('drive/My Drive/Colab Notebooks/data/user_ratings.csv')
path_to_master ='drive/My Drive/Colab Notebooks/data/predictions.csv'
if Path(path_to_master).is_file():
  old_master= pd.read_csv(path_to_master)
else:
  old_master = None

In [ ]:
algo = SVD()
reader = Reader(rating_scale=(1, 10))

In [ ]:
# have to give each movie and username a unique id 
user_df['user_id']= pd.factorize(user_df.lb_username)[0]+1 #I really have to save this user 
user_df['movie_id']= pd.factorize(user_df.Title)[0]+1
user_df.shape

(100683, 5)

In [ ]:
print(user_df['lb_username'].nunique())

38


In [ ]:
# print(user_df['user_id'].nunique())

38


In [ ]:
data=Dataset.load_from_df(user_df[['user_id','movie_id','User_Rating']], reader)

In [ ]:
#cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

In [ ]:
trainSet = data.build_full_trainset()

In [ ]:
algo.fit(trainSet)

In [ ]:
def make_pred(uid):
  """
  make predictions for one user.
  needs int of user
  returns predictions
  """
  #print('making predictions')
  anti_testset_user = []              #create empty DF
  targetUser = uid                    
  fillValue=trainSet.global_mean      #have to fill missing values with something. I decided on global mean for now
  user_item_ratings=trainSet.ur[targetUser]
  user_items = [item for (item,_) in (user_item_ratings)] #here we have all the ratings from that user
  ratings = trainSet.all_ratings()
  for iid in trainSet.all_items():    #surprise generator function to iterate over all items
    if(iid not in user_items):
      anti_testset_user.append((trainSet.to_raw_uid(targetUser),trainSet.to_raw_iid(iid),fillValue))
  predictions = algo.test(anti_testset_user)
  # print(fillValue) #just out of curiosity
  return predictions

In [72]:
def sort_into_master(pred, pred_master_new,user_id):
  user_id = user_id+1
  print(user_id)
  df=pred_master.copy(deep=True)
  user= user_df.loc[user_df['user_id'] == user_id,'lb_username'].iloc[0]
  #print('sorting ' + user+ 'into master df')
  #go over each movie(iid) of the predicted DF. put the est for that user on the fitting spot(next to the movie)
  for iid in pred['iid']:
    rating_temp = pred.loc[pred.loc[pred['iid']==iid].index.values[0], 'est'] #so the position has to be found we are looking for the 'est' of the current iid in the prediction DF now we have the est as a temp which we will save into the master DF
    df.loc[df['movie_id']==iid, user] =rating_temp                  #seems like this is the more clean solution. dont need an temp  index for it

  return df

In [ ]:
pred_master = user_df[['Title', 'movie_id']]
pred_master = pred_master.drop_duplicates(subset=['Title']) # first we drop duplicates. because this df is based on the user df it has duplicate movies
pred_master=pred_master.reset_index(drop=True)

In [75]:
username = 'swaiborr'
user_id= user_df.loc[user_df['lb_username'] == username,'user_id'].iloc[0]-1 #-1 because i have +1 in my sort into master ( i have that there because )
user_id

30

In [76]:
predictions = make_pred(user_id)  ## custom function which creates anti_testset of 1 user, makes predictions for that user
pred = pd.DataFrame(predictions) #put predictions in a DF which with we can work
pred.sort_values(by=['iid'],inplace=True,ascending = True)
#print('user id ' + str(user_id))
pred_temp = sort_into_master(pred, pred_master, user_id)
pred_swaiborr = pred_temp

31


In [82]:
pred_swaiborr['prediction'] = pred_swaiborr['swaiborr']
pred_swaiborr = pred_swaiborr.drop('swaiborr',axis=1)

In [97]:
pred_swaiborr['Title'].head()

1595                        Lady Bird
506                The Social Network
7336                          Stalker
4622         The Silence of the Lambs
3661    Twin Peaks: Fire Walk with Me
Name: Title, dtype: object

In [91]:
pred_swaiborr.sort_values(by=['prediction'],inplace=True,ascending = False)

In [99]:
recommendations = pred_swaiborr[['Title','prediction']].head(20) #get the first 20 entries for recommendations
print(recommendations)

                              Title  prediction
1595                      Lady Bird    4.661904
506              The Social Network    4.618121
7336                        Stalker    4.614533
4622       The Silence of the Lambs    4.613595
3661  Twin Peaks: Fire Walk with Me    4.611639
7296                          Yi Yi    4.610677
4874           Inglourious Basterds    4.602510
5670              Stop Making Sense    4.601483
3011             Mad Max: Fury Road    4.590376
974      Portrait of a Lady on Fire    4.578213
1442                    Shoplifters    4.543491
3545                      Gone Girl    4.540415
4514                  Spirited Away    4.536904
1813               Mulholland Drive    4.534052
1505                 First Reformed    4.527676
2511                          Carol    4.525250
3442                   Barry Lyndon    4.507114
4407          2001: A Space Odyssey    4.501528
5604           In the Mood for Love    4.499509
1002                 The Lighthouse    4

In [79]:
pred_swaiborr.to_csv('drive/My Drive/Colab Notebooks/data/prediction_swaiborr.csv',index=False, header=True)

so i checked my recommendations (for my user on letterboxd). I noticed that I didnt rate Dune which is a great example. I really liked the movie back then and would say that I would rate it a 4.5 or 5. the algo expects a rating of 3.92 as of today (4.9.2022) If we look at the dune movie it has a rating of 4.0. my average rating right now is 3.84. so this puts the rating right in the middle between my average and the average of the movie overall which seems fitting